# Model Selection

In this notebook, we will train models with the parameters gotten from hiperparametrization, compare the performance of said models and select and train the model that will be used for deployment

In [9]:
import pickle
from sklearn import metrics
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

In [6]:
filename = "../Models/ML/Hyperparametrized/hyper_models.pkl"
with open(filename, "rb") as file:
    mlp_grid, xgb_grid, rf_grid, mlp_bayes, xgb_bayes, rf_bayes = pickle.load(file)